In [68]:
# Imports
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision.datasets as datasets # pytorch has a lot of datasets # https://docs.pytorch.org/vision/stable/datasets.html
import torchvision.transforms as transforms

In [69]:
# Create Simple Fully Connected Network
class NN(nn.Module):
    def __init__(self, input_size, num_classes): # (28*28 = 784)
        super(NN, self).__init__()
        self.fc1 = nn.Linear(input_size, 50)
        self.fc2 = nn.Linear(50, num_classes)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [70]:
# TODO: Create simple CNN
class CNN(nn.Module):
    def __init__(self, in_channels = 1, num_classes = 10):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=8, kernel_size=(3,3), stride=(1,1), padding=(1,1)) # same convolution
        self.pool = nn.MaxPool2d(kernel_size=(2,2), stride=(2,2))    
        self.conv2 = nn.Conv2d(in_channels=8, out_channels=16, kernel_size=(3,3), stride=(1,1), padding=(1,1))
        self.fc1 = nn.Linear(16*7*7, num_classes)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        x = x.reshape(x.shape[0], -1)
        x = self.fc1(x)

        return x

In [71]:
# Save Checkpoint
def save_checkpoint(state, filename="models/my_checkpoint.pth.tar"):
    print("=> Saving checkpoint")
    torch.save(state, filename)

In [72]:
# Load Checkpoint
def load_checkpoint(checkpoint):
    print("=> Loading checkpoint")
    model.load_state_dict(checkpoint['state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer'])

In [73]:
# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [74]:
# Hyperparameters
in_channel = 1
num_classes = 10
learning_rate = 1e-4 # 0.001
batch_size = 1024
num_epochs = 10
load_model = True

In [75]:
# Load Data
train_dataset = datasets.MNIST(root='dataset/', train=True, transform=transforms.ToTensor(), download=True)
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)

test_dataset = datasets.MNIST(root='dataset/', train=False, transform=transforms.ToTensor(), download=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True)

In [76]:
# Initialize network
model = CNN().to(device)

In [77]:
# loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [78]:
# load model
if load_model:
    load_checkpoint(torch.load("models/my_checkpoint.pth.tar"))

=> Loading checkpoint


In [79]:
# Train network

for epoch in range(num_epochs):
    losses = []
    # if epoch == 2:
    if epoch % 3 == 0:
        checkpoint = { 'state_dict' : model.state_dict(), 'optimizer': optimizer.state_dict() }
        save_checkpoint(checkpoint)
    for batch_idx, (data, targets) in enumerate(train_loader):
        # Get data to cuda if possible
        data = data.to(device=device)
        targets = targets.to(device=device)

        # forward
        scores = model(data)
        loss = criterion(scores, targets)
        losses.append(loss.item())

        # backward
        optimizer.zero_grad()
        loss.backward()

        # gradient descent or adam step
        optimizer.step()

    mean_loss = sum(losses) / len(losses)
    print(f'Loss at epoch {epoch} was {mean_loss:.5f}')

=> Saving checkpoint
Loss at epoch 0 was 0.22771
Loss at epoch 1 was 0.22255
Loss at epoch 2 was 0.21779
=> Saving checkpoint
Loss at epoch 3 was 0.21289
Loss at epoch 4 was 0.20823


KeyboardInterrupt: 

In [14]:
# Check accuracy on training & test to see how good our model

def check_accuracy(loader, model):
    if loader.dataset.train:
        print("Checking accuracy on training data")
    else:
        print("Checking accuracy on test data")
    num_correct = 0
    num_samples = 0
    model.eval()

    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device)
            y = y.to(device=device)

            scores = model(x)
            scores.max()
            _, predictions = scores.max(1)
            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0)

        print(f'Got {num_correct} / {num_samples} with accuracy {float(num_correct)/float(num_samples)*100:.2f}')

    model.train()

check_accuracy(train_loader, model)
check_accuracy(test_loader, model)

Checking accuracy on training data
Got 59220 / 60000 with accuracy 98.70
Checking accuracy on test data
Got 9843 / 10000 with accuracy 98.43
